In [99]:
%profile profile_for_article
%iam_role arn:aws:iam::405389362913:role/iamr-glueintsessionsdemo
%idle_timeout 10
%number_of_workers 2

Previous profile: profile_for_article
Setting new profile to: profile_for_article
Current iam_role is arn:aws:iam::405389362913:role/iamr-glueintsessionsdemo
iam_role has been set to arn:aws:iam::405389362913:role/iamr-glueintsessionsdemo.
Current idle_timeout is 10 minutes.
idle_timeout has been set to 10 minutes.
Previous number of workers: 2
Setting new number of workers to: 2


In [101]:
%%configure
{
    "--s3_bucket_name" : "s3-glueintsessionsdemo-data"
}

The following configurations have been updated: {'--s3_bucket_name': 's3-glueintsessionsdemo-data'}


In [103]:
%%configure
{
    "--datalake-formats" : "iceberg",
    "--conf" : "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://s3-glueintsessionsdemo-data/iceberg/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO"
}

The following configurations have been updated: {'--datalake-formats': 'iceberg', '--conf': 'spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://s3-glueintsessionsdemo-data/iceberg/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO'}


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 2
Session ID: 63d3d32b-262a-4c7d-a5fd-074e4a9f3a4b
Applying the following default arguments:
--glue_kernel_version 1.0.0
--enable-glue-datacatalog true
--S3_BUCKET_NAME s3-glueintsessionsdemo-data
--datalake-formats iceberg
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.sql.catalog.glue_catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.glue_catalog.warehouse=s3://s3-glueintsessionsdemo-data/iceberg/ --conf spark.sql.catalog.glue_catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.glue_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO
--s3_bucket_name s3-glueintsessionsdemo-data
Waiting for session 63d3d32b-262a-4c7d-a5fd-074e4a9f3a4b to get into ready status...
Session 63d3d32b-262a-4c7d-a5fd-074e4a9f3a4b has been created.



In [2]:
from awsglue.utils import getResolvedOptions
parameter_name = 's3_bucket_name'
args = getResolvedOptions(sys.argv,[parameter_name])
s3_bucket_name = args[parameter_name]

In [3]:
# 1. Read dataframe directly from s3 object using the variable
df = spark.read.option("header", "true").option("inferSchema", "true").csv(f"s3://{s3_bucket_name}/in/lakes/")

In [ ]:
# 2. Convert it into parquet format and writes to specified S3 path using the variable
df.write.mode("overwrite").parquet(f"s3://{s3_bucket_name}/out/lakes/")

In [ ]:
# 3. Prepare aggregated dataframe
agg_df = df.groupBy("continent").count().withColumnRenamed("count", "number_of_lakes")

# Outputs dataframe
agg_df.show()

In [ ]:
# 4. Write aggregated dataframe to Iceberg table
agg_df.createOrReplaceTempView("tmp_lakes")

query = f"""
CREATE TABLE IF NOT EXISTS glue_catalog.default.lakes_iceberg
USING iceberg
AS SELECT * FROM tmp_lakes
"""
spark.sql(query)

In [ ]:
%stop_session